In [47]:
import pandas as pd
import numpy as np
from federated_learning import FederatedLearning, LogisticModel, visualization_weights
import matplotlib.pyplot as plt

In [48]:
def run_federated_learning(file_paths, features=['DriverAge', 'Gender', 'VehiculeUsage'], target='Sinistre', num_rounds=5):
    """
    Exécute l'apprentissage fédéré sur les données fournies.
    
    Args:
        file_paths: Dictionnaire {nom_client: chemin_fichier_csv}
        features: Liste des caractéristiques à utiliser
        target: Nom de la variable cible
        num_rounds: Nombre de cycles d'entraînement
        
    Returns:
        global_weights: Les poids finaux du modèle global
        clients_weights: Dictionnaire des poids de chaque client
    """
    try:
        # Charger les données
        data_dict = {}
        for client_name, path in file_paths.items():
            data_dict[client_name] = pd.read_csv(path)
            print(f"Données chargées pour {client_name}: {len(data_dict[client_name])} observations")
        
        # Normaliser les variables numériques
        for client_name, df in data_dict.items():
            if 'DriverAge' in df.columns:
                print(f"Normalisation de DriverAge pour {client_name}")
                df['DriverAge'] = (df['DriverAge'] - df['DriverAge'].mean()) / df['DriverAge'].std()
        
        # Configurer le système
        print("\nConfiguration du système d'apprentissage fédéré...")
        fl_system = FederatedLearning(
            data_dict=data_dict,
            features=features,
            target=target,
            model_class=LogisticModel,
            max_iter=1000,
            random_state=42,
            class_weight='balanced'
        )
        fl_system.setup()
        
        # Lancer l'entraînement
        print("\nDémarrage de l'entraînement...")
        global_weights = fl_system.train(num_rounds=num_rounds)
        
        print("\nApprentissage fédéré terminé!")
        print("Poids finaux du modèle global:")
        feature_names = ['Intercept'] + features
        for name, weight in zip(feature_names, global_weights):
            print(f"  {name}: {weight:.6f}")
        
        # Récupérer les poids de chaque client
        clients_weights = {}
        for client in fl_system.server.clients:
            clients_weights[client.name] = client.model.get_weights()
            
        return global_weights, clients_weights, feature_names
    
    except Exception as e:
        print(f"Erreur lors de l'exécution: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None


In [49]:

def main():
    
    # Chemins des fichiers
    file_paths = {
        'France': 'data/french_data.csv',
        'Belgium': 'data/belgium_data.csv',
        'Europe': 'data/european_data.csv'
    }
    
    # Caractéristiques à utiliser
    features = ['DriverAge', 'Gender', 'VehiculeUsage']
    
    # Lancer l'apprentissage fédéré
    global_weights, clients_weights, feature_names = run_federated_learning(
        file_paths=file_paths,
        features=features,
        target='Sinistre',
        num_rounds=5
    )
    
    if global_weights is not None:
        # Visualiser les poids du modèle global
        fig1 = visualization_weights.visualize_model_weights(global_weights, feature_names)
        plt.savefig('global_model_weights.png', dpi=300, bbox_inches='tight')
        plt.close(fig1)
        
        # Comparer les poids de tous les modèles
        if clients_weights:
            all_weights = [clients_weights[client] for client in clients_weights] + [global_weights]
            model_names = list(clients_weights.keys()) + ['Global']
            fig2 = visualization_weights.compare_model_weights(all_weights, model_names, feature_names)
            plt.savefig('models_comparison.png', dpi=300, bbox_inches='tight')
            plt.close(fig2)
        
        print("\nLes visualisations ont été sauvegardées dans 'global_model_weights.png' et 'models_comparison.png'")


In [50]:

if __name__ == "__main__":
    main()

Données chargées pour France: 243065 observations
Données chargées pour Belgium: 163212 observations
Données chargées pour Europe: 237319 observations
Normalisation de DriverAge pour France
Normalisation de DriverAge pour Belgium
Normalisation de DriverAge pour Europe

Configuration du système d'apprentissage fédéré...
Configuration du client 'France' avec 243065 observations
Client 'France' ajouté avec succès
Configuration du client 'Belgium' avec 163212 observations
Client 'Belgium' ajouté avec succès
Configuration du client 'Europe' avec 237319 observations
Client 'Europe' ajouté avec succès

Démarrage de l'entraînement...

Démarrage de l'apprentissage fédéré avec 3 clients

Poids du modèle local pour France:
  Intercept: 0.496634
  DriverAge: -0.062988
  Gender: -0.003088
  VehiculeUsage: -0.206355
Performance du modèle pour France:
  Accuracy: 0.0649
  Precision: 0.0649
  Recall: 1.0000
  F1: 0.1218

--- Cycle d'apprentissage fédéré 1/5 ---

Poids du modèle local pour France:
  In